## Example: American Put Option

### Import Packages

In [1]:
import pandas as pd
import numpy as np
import csv

### Initialize Parameters

In [2]:
# Total day
N = 15
# Initial price
init_price = 50
# Strike Price
strike_price = 48
# Chance of going up
chance_up = 0.6
# Chance of going down
chance_down = 0.4 
# Percentage of going up
percentage_up = 0.01
# Percentage of going down
percentage_down = 0.01 

### Initialize Tables

In [3]:
# Initialize value function and policy tables
# which are both (2N-1) * N matrix
stock_price_table = [[0 for d in range(N)] for d in range(2*N-1)]
value_function_table = [[0 for d in range(N)] for d in range(2*N-1)]
policy_table = [["" for d in range(N)] for d in range(2*N-1)]


# Initialize the stock price table
def intialize_table():
        for price in range(2 * N - 1):
            if price < N - 1:
                for period in range(N - 1 - price, N):
                    stock_price_table[price][period] = init_price * ((1 + percentage_up)**(N-1-price))
#                 for period in range(price+1):
#                     stock_price_table[price][N - 1 - period] = init_price * ((1 + percentage_up)**(N-1-price))
            
            if price == N - 1:
                for period in range(price + 1):
                    stock_price_table[price][period] = init_price
            
            if price > N - 1:
                for period in range(price - N + 1, N):
                    stock_price_table[price][period] = init_price * ((1 - percentage_down)**(price-N+1))
#                 for period in range(2*N-1 - price):
#                     stock_price_table[price][N - 1 - period] = init_price * ((1 - percentage_down)**(price-N+1))
        return

    
def value_iteration():
    # Initialize the last column first (at period = N - 1)
    for price in range(2*N-1):
            exercise = strike_price - stock_price_table[price][N-1]
            wait = 0
            value_function_table[price][N-1] = max(wait, exercise)
            if(wait > exercise):
                policy_table[price][N-1] = "wait"
            else:
                policy_table[price][N-1] = "exercise"
    
    # Initialize the rest by backward induction
    for period in range(N - 2,-1,-1):
        for price in range(N - period - 1, N + period):
            exercise = strike_price - stock_price_table[price][period]
            wait = chance_up * value_function_table[price - 1][period + 1] + chance_down * value_function_table[price + 1][period + 1]
            value_function_table[price][period] = max(wait, exercise)
            if(wait > exercise):
                policy_table[price][period] = "wait"
            else:
                policy_table[price][period] = "exercise"
            
    return

intialize_table()
value_iteration()

In [4]:
col_name = map(str, range(1, N+1))
rows = list(np.array(np.matrix(stock_price_table)[:,N-1]).reshape(-1,))
round_rows = list(np.around(np.array(rows),2))

### Value Function Table

In [5]:
value = pd.DataFrame(np.matrix(value_function_table))
value.columns = col_name
value.columns.name = 'Price/Period'
value.index = round_rows

value.to_csv("value_table.csv", sep=';') 

def color_negative_red1(val):
    color = 'red' if val > 0 else 'black'
    return 'color: %s' % color
value = value.style.applymap(color_negative_red1)
value

Price/Period,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
57.47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
56.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
56.34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55.78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55.23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54.68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54.14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53.61,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53.08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52.55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Policy Table

In [9]:
policy = pd.DataFrame(np.matrix(policy_table))
col_name = map(str, range(1, N+1))
policy.columns = col_name
policy.index = round_rows
policy.columns.name = 'Price/Period'
policy.to_csv("policy_table.csv", sep=';') 
policy

Price/Period,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
57.47,,,,,,,,,,,,,,,wait
56.90,,,,,,,,,,,,,,wait,wait
56.34,,,,,,,,,,,,,wait,wait,wait
55.78,,,,,,,,,,,,wait,wait,wait,wait
55.23,,,,,,,,,,,wait,wait,wait,wait,wait
54.68,,,,,,,,,,wait,wait,wait,wait,wait,wait
54.14,,,,,,,,,wait,wait,wait,wait,wait,wait,wait
53.61,,,,,,,,wait,wait,wait,wait,wait,wait,wait,wait
53.08,,,,,,,wait,wait,wait,wait,wait,wait,wait,wait,wait
52.55,,,,,,wait,wait,wait,wait,wait,wait,wait,wait,wait,wait


In [7]:
# Starts at day 1 with initial stock price 50
# value_iteration(0, 50)

In [8]:
# import pandas as pd
# #Parameters
# N = 5 # Total day
# chance_up = 0.6 # Chance of going up
# chance_down = 0.4 # Chance of going down
# percentage_up = 0.01 # Percentage of going up
# percentage_down = 0.01 # Percentage of going down

# action = ["" for d in range(N)] # Initialize action array
# value = [0 for d in range(N)] # Initialize value array
# wait = [0 for d in range(N)]
# exercise = [0 for d in range(N)]
# optimal_value = [0 for d in range(N)]

# def value_iteration(day, price):
    
#     # Calcululate the price of exercise
#     if (price >= 48):
#         exercise_value = 0
#     else:
#         exercise_value = 48 - price
    
#     # Calculate the price of wait
#     if day < N-1:
#         wait_value = chance_up * value_iteration(day + 1, 1.01*price) + chance_down * value_iteration(day + 1, 0.99*price)
#         wait[day] = wait_value
#         exercise[day] = exercise_value
        
#         # Decides the optimal action
#         if (wait_value > exercise_value):
#             action[day] = "wait"
#             optimal_value[day] = wait_value
#         else:
#             action[day] = "exercise"
#             optimal_value[day] = exercise_value
    
#     # The last day
#     if day == N-1:
#         action[day] = "exercise"
#         wait[day] = 0
#         exercise[day] = exercise_value
#         optimal_value[day] = exercise_value
        
#     return optimal_value[day]
